This notebook is a part of motokimura's baseline solution for the [Solafune Identifying Deforestation Drivers competition](https://solafune.com/competitions/68ad4759-4686-4bb3-94b8-7063f755b43d?menu=about&tab=overview).
See https://github.com/motokimura/solafune_deforestation_baseline for the complete code.

> cf. @solafune (https://solafune.com) Use for any purpose other than participation in the competition or commercial use is prohibited. If you would like to use them for any of the above purposes, please contact us.

### Description

This notebook generates segmentation masks for the training images and save them as `.npy` files.
These files are used as training labels (see https://github.com/motokimura/solafune_deforestation_baseline for the training code).

Each `.npy` file contains a numpy array of shape (4, 1024, 1024) for the four classes (`grassland_shrubland`, `logging`, `mining`, and `plantation`).
The pixels with value 255 are considered to be the corresponding class.

```
data/
├── train_masks/
│   ├── train_0.npy
│   ├── train_1.npy
│   ├── train_2.npy
│   ├── ...
```

The notebook also saves the visualization of the masks along with the RGB image as a png file.
These files are just for visualization (not used for training the model).

```
data/
├── vis_train/
│   ├── train_0.png
│   ├── train_1.png
│   ├── train_2.png
│   ├── ...
```

### Requirements

#### Datasets

Download the datasets and organize them as follows:

```
data/
├── evaluation_images/
│   ├── evaluation_0.tif
│   ├── evaluation_1.tif
│   ├── evaluation_2.tif
│   ├── ...
├── train_images/
│   ├── train_0.tif
│   ├── train_1.tif
│   ├── train_2.tif
│   ├── ...
├── train_annotations.json
```

#### Libraries

Please install the python packages imported the cell below.


In [1]:
import json
from pathlib import Path

import cv2
import numpy as np
import tifffile
from tqdm import tqdm

OpenCV bindings requires "numpy" package.
Install it via command:
    pip install numpy


ModuleNotFoundError: No module named 'numpy.core.multiarray'

In [2]:
import json
from pathlib import Path

import cv2
import numpy as np
import tifffile
from tqdm import tqdm

In [ ]:
import sys
from pathlib import Path

# Move up from the notebook directory to project root
BASE_DIR = Path.cwd().resolve().parent.parent  # Moves up two levels to `solafune-deforestation`
sys.path.insert(0, str(BASE_DIR))  # Add project root to Python path

# Import paths dynamically
from src.paths import DATASET_PATH, TRAIN_IMAGES_PATH, EVAL_IMAGES_PATH, TRAIN_ANNOTATIONS_PATH, VISUALIZATIONS_PATH

print("DATASET_PATH:", DATASET_PATH)  # Debugging


DATASET_PATH: C:\Users\Brage\Desktop\skole\solafune-deforestation\data


In [4]:
train_file_names = [f"train_{i}.tif" for i in range(176)]  # train_0.tif ~ train_175.tif
class_names = ["grassland_shrubland", "logging", "mining", "plantation"]

# Ensure DATASET_PATH is a Path object before using "/"
DATASET_PATH = Path(DATASET_PATH)  # Convert string to Path
with open(DATASET_PATH / "train_annotations.json", "r") as f:
    raw_annotations = json.load(f)

annotations: dict[str, dict[str, list[list[float]]]] = {}  # file_name -> class_name -> polygons
for fn in tqdm(train_file_names):
    ann: dict[str, list[list[float]]] = {}  # class_name -> polygons
    for class_name in class_names:
        ann[class_name] = []

    for tmp_img in raw_annotations["images"]:
        if tmp_img["file_name"] == fn:
            for tmp_ann in tmp_img["annotations"]:
                ann[tmp_ann["class"]].append(tmp_ann["segmentation"])

    annotations[fn] = ann

#print(annotations["train_0.tif"])
# {'grassland_shrubland': [], 'logging': [], 'mining': [], 'plantation': [[0.0, 449.0, 9.0, 454.0, 18.0, 461.0, 26.0, 468.0, 33.0, 475.0, 40.0, 477.0, ...

100%|██████████| 176/176 [00:00<00:00, 29280.77it/s]


In [5]:
mask_save_dir = DATASET_PATH / "train_masks"
mask_save_dir.mkdir(parents=True, exist_ok=True)

for fn in tqdm(train_file_names):
    mask = np.zeros((4, 1024, 1024), dtype=np.uint8)
    anns = annotations[fn]
    for class_idx, class_name in enumerate(class_names):
        polygons = anns[class_name]
        cv2.fillPoly(mask[class_idx], [np.array(poly).astype(np.int32).reshape(-1, 2) for poly in polygons], 255)

    np.save(mask_save_dir / fn.replace(".tif", ".npy"), mask)

100%|██████████| 176/176 [00:00<00:00, 221.14it/s]


In [ ]:
# visualize masks and save as a png file along with the RGB image

vis_save_dir = VISUALIZATIONS_PATH / "vis_train"
vis_save_dir.mkdir(parents=True, exist_ok=True)

for fn in tqdm(train_file_names):
    mask = np.load(mask_save_dir / fn.replace(".tif", ".npy"))  # (4, 1024, 1024)
    vis_masks = [np.zeros((1024, 1024, 3), dtype=np.uint8) for _ in range(4)]  # 4: (glassland_shrubland, logging, mining, plantation)
    for class_idx, class_name in enumerate(class_names):
        vis_masks[class_idx][mask[class_idx] > 0] = np.array([255, 0, 0])  # blue
        # put class_name as text on the mask
        cv2.putText(vis_masks[class_idx], class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    vis_image = tifffile.imread(DATASET_PATH / "train_images" / fn)
    vis_image = vis_image[:, :, [1, 2, 3]]  # extract BGR channels (B2, B3, and B4 band of Sentinel-2)
    vis_image = np.nan_to_num(vis_image, nan=0)
    vis_image = (vis_image / 8).clip(0, 255).astype(np.uint8)

    partition = np.ones((1024, 5, 3), dtype=np.uint8) * 255  # white partition
    vis = np.concatenate([vis_image, partition, vis_masks[0], partition, vis_masks[1], partition, vis_masks[2], partition, vis_masks[3]], axis=1)
    cv2.imwrite(str(vis_save_dir / fn.replace(".tif", ".png")), vis)


100%|██████████| 176/176 [01:16<00:00,  2.29it/s]
